In [2]:
import numpy as np
import pandas as pd

In [3]:
# what varies for all data sets
# n_classes_dep_var = 
# n_ind_vars =
# n_classes_ind_vars = 
# n_samples = 

def generate_data(n_classes_dep_var, n_ind_vars, n_classes_ind_vars, n_samples):
    # set coefficients for the model
    coef = [3, 2, 2, 1, 1, 0.5, 0.5]
    coef = coef[:n_ind_vars]

    # generate n x p array for independent variables (predictors)
    X = np.random.uniform(0, 1, (n_samples, n_ind_vars))

    # generate error term
    e = np.random.normal(0, 1, n_samples)

    # calculate y continuous
    y_continuous = 1 + np.dot(X, coef) + e

    # convert y to categorical
    # keep working from here
    if n_classes_dep_var == 2:
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.5), 'A', 'B')
    elif n_classes_dep_var == 3: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous > np.quantile(y_continuous, 0.75), 'B', 'C'))
    elif n_classes_dep_var == 4: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.5), 'B',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.75), 'C', 'D')))

    # create data that is to be returned
    data_mat = np.concatenate((X, y_continuous.reshape(-1, 1), y_categorical.reshape(-1, 1)), axis = 1)
    data = pd.DataFrame(data_mat)
    # Rename columns to X1, X2, ..., y_cont, y_cat
    data.columns = [f'X{i+1}' for i in range(n_ind_vars)] + ['y_cont', 'y_cat']

    return(data)


In [4]:
# try an example
data = generate_data(3, 3, 0, 100)
print(data)

                     X1                   X2                    X3  \
0    0.7001170636665754   0.5076038832211137   0.12386117662686369   
1    0.4178859561810945   0.5191980626711906    0.7969823911358124   
2   0.04219673948009661   0.6294299679662735    0.9311003408679961   
3     0.626323577505594  0.22107418519469724    0.9319170919312793   
4    0.6504474681170278  0.13651485665620722    0.7458740244777678   
..                  ...                  ...                   ...   
95  0.07348328032112317  0.08267748630213345   0.08202088722644851   
96   0.9094703729561132   0.7391294928951823  0.005410062887529432   
97   0.8212865112113674  0.15828307417479037    0.2250969445949288   
98   0.7915002477139209   0.2889854522278008    0.6840415632859603   
99   0.6917067406620524  0.44233103537997187   0.22383805966807568   

                y_cont y_cat  
0   3.9388930882014983     C  
1    5.522436586759499     C  
2    3.561565420190478     C  
3    5.256654929063483     C  
4   

In [6]:
n_classes_dep_var = [2, 3, 4]
n_ind_vars = [3, 5, 7]
n_classes_ind_vars = np.arange(6)
n_samples = [100, 500, 1000, 10000]